In [10]:
import pandas as pd

strDate = '20200801'
endDate = '20210101'

dt_index = pd.date_range(start=strDate, end=endDate)
dt_list = dt_index.strftime("%Y. %m. %d.").tolist()

for i in dt_list:
    print(f"SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '{i}%'")

SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 01.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 02.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 03.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 04.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 05.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 06.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 07.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 08.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 09.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 10.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 11.%'
SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 12.%'
SELECT pubdate, news_content

In [20]:
import re
import sqlite3
from konlpy.tag import Okt

# !python -m pip install konlpy

okt = Okt()
def cutword(i):
  # stopwords = ['코로나', '올림픽', '폐렴', '바이러스', '백신', '수급', '캠페인', '은', '도', '는', '다', '사', '의', '위', '제', '회', '등', '가', '이', '했', '한', '에', '애', '하', '보', '웃', '고', '곧', '정', '적', '순', '뜻', '용', '열', '유', '뜻', '잘', '못', '타', '희', '히', '점', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '개', '것', '명', '원', '세','에서', '이고', '라고', '다고', '라기', '라며', '면서', '라면서', '로써', '로서', '로', '으로', '뿐', '에서', '어야', '부터', '었', '았', '한다', '이다', '였다', '였었다', '|', '/','‘', '’', ',' , '“', '”', '.', '>', '<', ')', '(']
  stopwords = []
  words = list()
  for wd in news_con[i]:
    temp = ''.join(wd)
    temp_re = re.sub('[^가-힣]','', temp)
    encoded = okt.morphs(temp_re)
    c_word = [item for item in encoded if item not in stopwords]
    words.append(c_word)

  return words

conn = sqlite3.connect('../2_daum_news_preprocessing/scrapping.db')
c = conn.cursor()
c.execute("SELECT pubdate, news_content FROM news_dummy WHERE pubdate LIKE '2020. 08. 01. 23:%'")

news_con = []
for row in c.fetchall():
  print(row)
  # news_con.append(row)

# for i in range(len(news_con)):
#   news_word = cutword(i)
#   print(news_word)

conn.close()

('2020. 08. 01. 23:59', 'British Formula One Grand Prix epa         British Formula One driver Lewis Hamilton of Mercedes AMG Petronas steps our of his car after he took pole position during the qualifying session of the British Formula One Grand Prix in Silverstone  Britain     August       EPA Will Oliver   Pool')
('2020. 08. 01. 23:59', 'German Federal Foreign Office issues travel warning for three Spanish regions epa         An information board shows departure flights to Palma de Majorca and Barcelona in Spain  at the International Airport in Duesseldorf  Germany     August       During the holiday season  new infections with the coronavirus are also increasingly occurring in several European countries  Because of the sharp rise in the number of corona infections in Spain  the Federal Foreign Office is now warning against tourist travel to three regions of Germany s most popular holiday destination  According to a statement by the Federal Foreign Office  Catalonia  with its touris